In [1]:
from elasticsearch import Elasticsearch, helpers

es = Elasticsearch()

In [2]:
import os
import json
import pprint

## Table Create

In [3]:
# db 이름 설정
INDEX_NAME = "news_wiki_index_update"

# db 셋팅
INDEX_SETTINGS = {
"settings" : {
    "index":{
    "analysis":{
        "analyzer":{
        "korean":{
            "type":"custom",
            "tokenizer":"nori_tokenizer",
            "filter": [ "shingle" ],

        }
        }
    }
    }
},
"mappings": {

    "properties" : {
        
        "admin_id" : {
        "type" : "keyword",
        },
        
        "category" : {
        "type" : "keyword",
        },
        
        "date" : {
        "type" : "date"
        },
        
        "title" : {
        "type" : "keyword",
        },
        
        "article" : {
        "type" : "text"
        },
        
        "context" : {
        "type" : "text",
        "analyzer": "korean",
        "search_analyzer": "korean"
        }
        
    }

}
}


In [31]:
def create_index(INDEX_NAME, INDEX_SETTINGS):
    if es.indices.exists(INDEX_NAME):
        es.indices.delete(index=INDEX_NAME)
    es.indices.create(index=INDEX_NAME, body=INDEX_SETTINGS)


In [32]:
create_index(INDEX_NAME, INDEX_SETTINGS)

## Table Insert

In [20]:
os.listdir('../data/')

['mbn_news_wiki.json',
 'mbn_news_wiki_past.json',
 'test_dataset',
 'train_dataset',
 'train']

In [21]:
with open(os.path.join("../data/", "mbn_news_wiki.json"), "r", encoding="utf-8") as f:
    wiki = json.load(f)

In [43]:
wiki['0']

{'admin_id': 'AIPaperboy',
 'article': '【 앵커멘트 】<br /><br />  지난해를 보내고 병신년 새해를 맞이하는 제야행사에도 많은 인파가 북적였습니다.<br /><br /> 수만 명의 시민들이 몰려들어 새해에 대한 설렘으로 소망을 기원했습니다.<br /><br />  오태윤 기자의 보도입니다.<br /><br /><br /><br /><br /><br />【 기자 】<br /><br />  차량이 통제되고 경찰들이 줄지어 들어오면서 차단막을 만듭니다. <br /><br /><br /><br />  그러자 수만 명의 시민이 도로 위로 모여듭니다.<br /><br /><br /><br />  2016년 병신년 새해의 보신각 타종을 보려는 인파입니다.<br /><br /><br /><br />  "3,2,1,0! 와!" <br /><br /><br /><br />▶ 인터뷰 : 이자연 / 서울 삼선동<br /><br />- "우리 가족 모두 건강하고 하는 일도 모두 잘됐으면 좋겠습니다."<br /><br /><br /><br />▶ 스탠딩 : 오태윤 / 기자<br /><br />- "영하의 날씨지만 제야의 종소리를 듣기 위해 7만여 명의 시민이 보신각 앞에 모였습니다."<br /><br /><br /><br />  만여 명의 인파가 몰린 강남 영동대로에선 싸이 등 인기가수들의 공연과 화려한 불꽃 쇼도 펼쳐졌습니다.<br /><br /><br /><br />▶ 인터뷰 : 김희성 / 인천 논현동<br /><br />- "원하는 곳에 취직지원서를 넣어서 바라는 대로 취직이 됐으면 좋겠습니다."<br /><br /><br /><br />  시민들은 새해에는 행복한 일이 가득하기를 바라는 마음을 담아 한목소리로 외쳤습니다.<br /><br /><br /><br />  새해 복 많이 받으세요.<br /><br /><br /><br />  새해 복 많이 받으세요.<br /><br /><br /><br />  새해 복 많이 받으세요.<br />

In [48]:
pprint.pprint(len(wiki['0']['article']))

1092


In [45]:
len(wiki.values())

842904

In [52]:
# 데이터 elastic search 입력 형식으로 변환
wikis =[]
for doc in wiki.values():
    temp = {
    "_index": INDEX_NAME,
    "_id" : doc['news_id'],
    "_source": {
        "admin_id":doc['admin_id'],
        "category":doc['category'],
        "date":doc['date'],
        "title":doc['title'],
        "article":doc['article'],
        "context": doc['context']
        }
    }
    wikis.append(temp)
#         wikis.append()

In [53]:
len(wikis)

842904

In [54]:
# 데이터 입력
try:
    for idx in range(1000,len(wikis), 1000):
        response = helpers.bulk(es, wikis[idx-1000:idx])
        # print ("\nRESPONSE:", response)
    response = helpers.bulk(es, wikis[842000:len(wikis)])
    print ("\nRESPONSE:", response)
except Exception as e:
    print("\nERROR:", e)


ERROR: ('1 document(s) failed to index.', [{'index': {'_index': 'news_wiki_index_update', '_type': '_doc', '_id': '3669882', 'status': 400, 'error': {'type': 'mapper_parsing_exception', 'reason': 'failed to parse', 'caused_by': {'type': 'json_parse_exception', 'reason': "Non-standard token 'NaN': enable JsonParser.Feature.ALLOW_NON_NUMERIC_NUMBERS to allow\n at [Source: (ByteArrayInputStream); line: 1, column: 77]"}}, 'data': {'admin_id': 'AIPaperboy', 'category': '연예', 'date': '2018-10-26', 'title': nan, 'article': '<!------------ PHOTO_POS_0 ------------> [ 매일경제 스타투데이 허은경 객원기자 ]<br />‘정글의 법칙’ 붐, 김도연, 루카스가 뉴페이스로 합류했다. <br />26일 방송된 SBS 예능 ‘정글의 법칙 in 라스트 인도양’(이하 ‘정글의 법칙’)에서는 스리랑카에 신입 병만 붐, 위키미키 김도연, NCT 루카스가 새내기로 합류한 모습이 전파를 탔다.<br />이날 병만족은 스리랑카 바다에 도착해 생존을 시작했다. 새롭게 합류한 위키미키 김도연은 "리더 기질이 있다. 고등학교 때 전교 회장도 하고, 치어리더 단장도 했다”면서 해피 바이러스를 담당하겠다"고 다짐을 전했다. 이후 김도연은 병만족 멤버들 앞에서 정글식 치어리딩을 선보여 감탄을 자아냈다.<br />NCT 루카스는 "중국에서 온 스무살 루카스다. 욕심부자다. 벌레, 모기 괜찮다. 귀신도 안 무섭다"며 정글생활에 열정을 드러냈다. 붐은 "어렸을 때 정글

In [55]:
def insert(body):
    return es.index(index=index, doc_type=doc_type, body=body)


'''
index = 'news'
doc_type = 'daum'
data = {
    'date' : '20200811',
    'category' : 'society',
    'title' : '유령'은 어느 행성에서 지구로 왔는가?'
    'content' : '크립톤 행성'
}

ir = insert(data)
'''

"\nindex = 'news'\ndoc_type = 'daum'\ndata = {\n    'date' : '20200811',\n    'category' : 'society',\n    'title' : '유령'은 어느 행성에서 지구로 왔는가?'\n    'content' : '크립톤 행성'\n}\n\nir = insert(data)\n"

## Table Read

In [61]:
res = es.get(index = INDEX_NAME, id='3719205')
res

{'_index': 'news_wiki_index_update',
 '_type': '_doc',
 '_id': '3719205',
 '_version': 1,
 '_seq_no': 163967,
 '_primary_term': 1,
 'found': True,
 '_source': {'admin_id': 'AIPaperboy',
  'category': '경제',
  'date': '2018-12-25',
  'title': "'2019 최저임금 산정'에 주휴시간 포함…야 4당 입장 엇갈려",
  'article': '<!------------ PHOTO_POS_0 ------------><br /><br />어제(24일) 정부가 최저임금 산정 기준시간에 주휴 시간을 포함하기로 하자 자유한국당·바른미래당·민주평화당·정의당 등 야 4당이  대해 입장이 엇갈렸습니다.<br /><br /><br /><br />약정 휴일을 최저임금 산정에서 빼기로 한 것과 관련해서도 야 4당이 대조된 입장을 밝혔습니다.<br /><br /><br /><br />여당인 더불어민주당은 별다른 입장을 내놓지 않으면서 오는 31일 국무회의 결과를 지켜볼 것으로 예상됩니다.<br /><br /><br /><br />보수 야권에 속하는 한국당과 바른미래당은 최저임금법 시행령 개정안에 대해 강하게 반발했습니다.<br /><br /><br /><br />한국당 윤영석 수석대변인은 "정부는 약정 휴일은 제외하기로 했다고 하지만, 기업의 약정 휴일이 1년에 며칠이나 되는지 의문"이라며 "주휴 시간은 포함하고 실효성 낮은 약정 휴일을 제외하는 것은 겉만 그럴싸하게 포장한 보여주기식 정책의 전형"이라고 비판했습니다.<br /><br /><br /><br />윤 수석대변인은 ""문재인정부가 신중한 고려 없이 내놓은 정책에 국민들의 고통만 가중되고 있다"고 말했습니다.<br /><br /><br /><br />그러면서 "최저임금 시행령 개악을 멈춰야 한다. 그렇지 않으면 주휴수당 최저임금 산입 저지는 한국

In [62]:
res['_source']['article']

'<!------------ PHOTO_POS_0 ------------><br /><br />어제(24일) 정부가 최저임금 산정 기준시간에 주휴 시간을 포함하기로 하자 자유한국당·바른미래당·민주평화당·정의당 등 야 4당이  대해 입장이 엇갈렸습니다.<br /><br /><br /><br />약정 휴일을 최저임금 산정에서 빼기로 한 것과 관련해서도 야 4당이 대조된 입장을 밝혔습니다.<br /><br /><br /><br />여당인 더불어민주당은 별다른 입장을 내놓지 않으면서 오는 31일 국무회의 결과를 지켜볼 것으로 예상됩니다.<br /><br /><br /><br />보수 야권에 속하는 한국당과 바른미래당은 최저임금법 시행령 개정안에 대해 강하게 반발했습니다.<br /><br /><br /><br />한국당 윤영석 수석대변인은 "정부는 약정 휴일은 제외하기로 했다고 하지만, 기업의 약정 휴일이 1년에 며칠이나 되는지 의문"이라며 "주휴 시간은 포함하고 실효성 낮은 약정 휴일을 제외하는 것은 겉만 그럴싸하게 포장한 보여주기식 정책의 전형"이라고 비판했습니다.<br /><br /><br /><br />윤 수석대변인은 ""문재인정부가 신중한 고려 없이 내놓은 정책에 국민들의 고통만 가중되고 있다"고 말했습니다.<br /><br /><br /><br />그러면서 "최저임금 시행령 개악을 멈춰야 한다. 그렇지 않으면 주휴수당 최저임금 산입 저지는 한국당의 제1의 경제 비상조치가 될 것"이라고 말했습니다.<br /><br /><br /><br />바른미래당 김삼화 수석대변인은 "문재인 정부는 눈이 있어도 보질 않고 귀가 있어도 듣질 않는 불통 행보로 최저임금과 주52시간 근무제, 주휴시간 산입을 밀어붙이고 있다"며 "단언컨대 내년 한국 경제는 극심한 경기침체와 고용 대란에 직면할 것"이라고 경고했습니다.<br /><br /><br /><br />이어 "경제 주체들의 역량을 고려하지 않고 강제로 임금을 높여 소상공인은 물론 기업들까지 타격을 받고, 이에 따른 물가상승과

### 카테고리 지정, 날짜 범위 지정, 쿼리 지정

In [63]:
def search(index, data=None):
    if data is None:
        data = {"match_all" : {}}
    else:
        data = {"match" : data}
    body = {"query" : data}
    res = es.search(index=index, body=body)
    
'''
sr = search(index, {'category': 'society'})
pprint.pprint(sr)
'''

"\nsr = search(index, {'category': 'society'})\npprint.pprint(sr)\n"

### 정렬하기, 호출 데이터 갯수 지정

In [64]:
body = {
  "query": {
    "bool": {
      "filter": [
        { "term": { "category": "정치"}},
        { "term": { "date": "2017-01-01" }}
      ]
    }    
    }
  }

query = "문재인이 시행한 정책은?"
es.search(index = INDEX_NAME, q = query, body = body)

 전국의사 총궐기대회를 차기 의협 회장과 협의해 강력하게 추진하고 전국의사 총파업 등 강경투쟁도 적극 검토하겠다"고 말했다.<br /> 이 같은 의협의 입장에 대해 주변 반응은 싸늘하다.<br /> 상복부 초음파 건보확대 적용은 오래 전에 예고돼 왔고 상복부 질환자 307여만명의 의료비 부담이 평균 6~16만원에서 2~6만원으로 경감되어 국민 대다수가 호의적이다. 일부 의료계 인사들도 초음파 급여 적용확대를 두고 장외집회를 여는 것은 "국민건강은 안중에 없는 또 하나의 집단이기주의"이라고 비판했다. 한 시민도 "의협이 국민의 건강을 위해 문재인케어 저지를 주장하지만 속내는 사익을 추구하는 것 아니냐"면서 "더이상 \'국민건강\'을 팔지 말고 그 시간에 환자를 돌보거나 공부하라"고 충고했다.<br />[이병문 의료전문 기자]<br /><br />[ⓒ 매일경제 & mk.co.kr, 무단전재 및 재배포 금지]<br><br /><br />',
     'context': '\'국민의 건강을 위한 것인가 아니면 국민건강 수호를 위장한 집단이기주의인가\' 대한의사협회가 18일 오후 1시 30분 서울 광화문에서 약 1000여명이 참가한 가운데 문재인케어 저지를 위한 \'전국의사 대표자대회\'를 개최한다고 15일 밝혔다. 이유는 정부가 의협 국민건강수호 비상대책위원회(비대위)와 협의없이 다음달부터 상복부 초음파 보험 적용범위를 전면 확대 시행한다고 밝혔기 때문이라고 했다. 의협은 "비대위와 정부가 9차례에 걸친 의정실무협의체에서 비대위와 사전협의를 통해 문재인 케어를 추진할 것이라고 지속적으로 말해왔음에도 예비급여를 일방 강행함으로써 비대위와의 파트너십을 무너뜨린 만큼, 이번 전국의사 대표자대회는 대정부 강경투쟁의 기폭제가 될 것"고 밝혔다. 의협은 특히 23일 예정된 회장선거에 출마한 추무진, 기동훈, 최대집, 임수흠, 김숙희, 이용민 후보 등 6명이 모두 참석할 것으로 보여 어느 때보다 긴장감이 고조될 전망이다. 의협회장 후보들은 모두 \'문재인케어가 우리나라 의료체계에

body = {
  "query": {
      "range": {
      "date": {
        "gt": "2016-01-01",
        "lt": "2016-01-01||+3M"
      }
    }
  }
  }

query = "문재인이 시행한 정책은?"
es.search(index = INDEX_NAME, q = query, body = body)

## Table Update

In [65]:
def update(id, doc):
    body = {'doc' : doc}
    res = es.update(index=index, id=id, body=body, doc_type=doc_type)
    return res

ir = insert(data)
ur = update(ir['_id'], {newspaper:'sbs'})
pprint.pprint(ur)

NameError: name 'data' is not defined

## Table Delete

In [ ]:
def delete_index(index):
    if not es.indices.exists(index=index):
        return es.indices.delete(index=index)

In [ ]:
def delete_by_id(index, id):
    return es.delete(index, id=id)